# #6 - Features (Spots & Gaussian Processes)

In this tutorial we'll learn how to add spots on a star and gaussian processes to a light curve.

This interactive workshop tutorial covers many of the same topics as the corresponding online tutorials:
* [Features](http://phoebe-project.org/docs/2.3/tutorials/features.ipynb)
* [Advanced: Spots](http://phoebe-project.org/docs/2.3/tutorials/spots.ipynb)
* [Example: Gaussian Processes](http://phoebe-project.org/docs/2.3/examples/minimal_GPs.ipynb)

# Setup

In [1]:
import phoebe
from phoebe import u,c

In [2]:
logger = phoebe.logger(clevel='WARNING')

In [3]:
b = phoebe.default_binary()

# Spots

Multiple spots can be attached to any given Star in the system.  Because of this, they live in the "feature" context (along with future features such as pulsations).  Adding a spot follows the same syntax as datasets or compute options:

In [4]:
b.add_feature('spot', component='primary', feature='spot01')

<ParameterSet: 4 parameters | qualifiers: colat, radius, long, relteff>

In [5]:
print(b.filter(feature='spot01'))

ParameterSet: 4 parameters
            colat@spot01@feature: 0.0 deg
             long@spot01@feature: 0.0 deg
           radius@spot01@feature: 1.0 deg
          relteff@spot01@feature: 1.0


As a shortcut, you can call b.add_spot:

In [6]:
b.add_spot(component='secondary', feature='spot02')

<ParameterSet: 4 parameters | qualifiers: colat, radius, long, relteff>

The spot is parameterized by the colatitude (where 0 is defined as the North (spin) Pole) and longitude (where 0 is defined as pointing towards the other star for a binary, or to the observer for a single star) of its center, its angular radius, and the ratio of temperature of the spot to the local intrinsic value.

# Exercise

Plot a light curve with and without a spot.

Make a mesh plot that shows the presence of a spot and get comfortable with how the coordinate system is defined.  Try for both binary systems and single stars.